[![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-181717?style=for-the-badge&logo=GitHub&link=https://github.com/Mearman/openalex-docs)](https://github.com/Mearman/openalex-docs)[![Open in GitHub](https://img.shields.io/badge/Open%20in-GitHub-181717?style=for-the-badge&logo=github&link=https://github.com/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/sort-entity-lists.ipynb)](https://github.com/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/sort-entity-lists.ipynb)[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=for-the-badge&logo=Google%20Colab&link=https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/sort-entity-lists.ipynb)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/how-to-use-the-api/get-lists-of-entities/sort-entity-lists.ipynb)

In [ ]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python-pydantic-v1.git"
%pip install pandasai

In [ ]:
import json
import pandas as pd
import numpy as np
from openalex_api import Configuration, ApiClient, AutocompleteApi, AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApi

configuration = Configuration(host="https://api.openalex.org")
autocomplete_api = AutocompleteApi(ApiClient(configuration))
authors_api = AuthorsApi(ApiClient(configuration))
concepts_api = ConceptsApi(ApiClient(configuration))
funders_api = FundersApi(ApiClient(configuration))
institutions_api = InstitutionsApi(ApiClient(configuration))
publishers_api = PublishersApi(ApiClient(configuration))
sources_api = SourcesApi(ApiClient(configuration))
works_api = WorksApi(ApiClient(configuration))

from pandasai import SmartDataframe
from pandasai.llm import OpenAI

In [ ]:
# @title  { run: "auto", display-mode: "form" }
openapi_token = "" # @param {type:"string"}

# Sort entity lists

Use the `?sort` parameter to specify the property you want your list sorted by. You can sort by these properties, where they exist:

* `display_name`
* `cited_by_count`
* `works_count`
* `publication_date`
* `relevance_score` (only exists if there's a [search filter](./sort-entity-lists.md#search) active)

By default, sort direction is ascending. You can reverse this by appending `:desc` to the sort key like `works_count:desc`. You can sort by multiple properties by providing multiple sort keys, separated by commas. Examples:

* All works, sorted by `cited_by_count` (highest counts first)\
  [`https://api.openalex.org/works?sort=cited_by_count:desc`](https://api.openalex.org/works?sort=cited\_by\_count:desc)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/works?sort=cited_by_count:desc
sort="cited_by_count:desc" # @param "cited_by_count:desc" {type: "string"}

response = works_api.get_works(
	sort=sort
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)

try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	if not openapi_token:
		print("Error: openapi_token not set")
	else:
		print("Error when creating SmartDataframe")

* All sources, in alphabetical order by title:\
  [`https://api.openalex.org/sources?sort=display_name`](https://api.openalex.org/sources?sort=display\_name)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/sources?sort=display_name
sort="display_name" # @param "display_name" {type: "string"}

response = sources_api.get_sources(
	sort=sort
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)

try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	if not openapi_token:
		print("Error: openapi_token not set")
	else:
		print("Error when creating SmartDataframe")

You can sort by relevance\_score when searching:

* Sort by year, then by relevance\_score when searching for "bioplastics":\
  [`https://api.openalex.org/works?filter=display_name.search:bioplastics&sort=publication_year:desc,relevance_score:desc`](https://api.openalex.org/works?filter=display\_name.search:bioplastics\&sort=publication\_year:desc,relevance\_score:desc)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/works?filter=display_name.search:bioplastics&sort=publication_year:desc,relevance_score:desc
filter="display_name.search:bioplastics" # @param "display_name.search:bioplastics" {type: "string"},
sort="publication_year:desc,relevance_score:desc" # @param "publication_year:desc,relevance_score:desc" {type: "string"}

response = works_api.get_works(
	filter=filter,
	sort=sort
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)

try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	if not openapi_token:
		print("Error: openapi_token not set")
	else:
		print("Error when creating SmartDataframe")

An error is thrown if attempting to sort by `relevance_score` without a search query.